# New dim_shapes

In [1]:
import datetime
import geopandas as gpd
import os
import pandas as pd

os.environ["CALITP_BQ_MAX_BYTES"] = str(130_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

import shared_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0326 23:09:12.918019963     948 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0326 23:09:16.503696946     948 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


## With string as date

In [4]:
SELECTED_DATE = "2022-1-1"

start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes_geo()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter((_.calitp_itp_id==17) | (_.calitp_itp_id==4))
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id, _.pt_array)
     )

end = datetime.datetime.now()
print(end - start)

0:00:00.284413


/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'


## With datetime as date

In [5]:
SELECTED_DATE = datetime.date(2022, 1, 1)

start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes_geo()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter((_.calitp_itp_id==17) | (_.calitp_itp_id==4))
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id, _.pt_array)
     )

end = datetime.datetime.now()
print(end - start)

0:00:00.247681


## Handle subsets of operators 

* By default, exclude `ITP_ID==200`, but what if we just want 5 operators?
* Instead of `collect` + `isin`, can we construct `or` statement?

In [2]:
start = datetime.datetime.now()

SELECTED_DATE = datetime.date(2022, 2, 8)

gdf = shared_utils.geography_utils.make_routes_gdf(
    SELECTED_DATE, CRS = "EPSG:2229", ITP_ID_LIST = [100, 182])
display(gdf.head())

end = datetime.datetime.now()
print(end - start)

,calitp_itp_id,calitp_url_number,shape_id,pt_array,geometry
0,182,0,1660113_DEC21,"[POINT(-118.5022048368 34.2357544125), POINT(-...","LINESTRING (6409877.000 1908556.800, 6411051.0..."
1,182,0,200782_DEC21,"[POINT(-118.4471901193 34.0570734142), POINT(-...","LINESTRING (6426215.600 1843453.600, 6426275.0..."
2,182,0,530227_DEC21,"[POINT(-118.2571788337 33.8672496709), POINT(-...","LINESTRING (6483591.900 1774171.200, 6484150.0..."
3,182,0,2330148_DEC21,"[POINT(-118.4468335137 34.151641553), POINT(-1...","LINESTRING (6426476.600 1877867.900, 6426471.0..."
4,182,0,1170072_DEC21,"[POINT(-118.1407682647 33.9130132985), POINT(-...","LINESTRING (6518955.300 1790755.700, 6518944.0..."


0:00:24.140429


In [3]:
start = datetime.datetime.now()

SELECTED_DATE = datetime.date(2022, 3, 15)

gdf = shared_utils.geography_utils.make_routes_gdf(
    SELECTED_DATE, CRS = "EPSG:3310", ITP_ID_LIST = None)
display(gdf.head())

end = datetime.datetime.now()
print(end - start)

/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


,calitp_itp_id,calitp_url_number,shape_id,pt_array,geometry
0,282,0,195174,"[POINT(-122.452613 37.723656), POINT(-122.4523...","LINESTRING (-215859.874 -29747.369, -215835.75..."
1,235,0,2604,"[POINT(-117.979775 33.85842), POINT(-117.98122...","LINESTRING (186958.516 -459878.144, 186824.025..."
2,290,1,SFO0024,"[POINT(-122.386624 37.599452), POINT(-122.3867...","LINESTRING (-210400.815 -43698.495, -210411.25..."
3,120,0,5sb,"[POINT(-118.2696925 34.16519183), POINT(-118.2...","LINESTRING (159510.789 -426386.226, 159484.409..."
4,61,0,43994,"[POINT(-121.96038 37.77031), POINT(-121.9608 3...","LINESTRING (-172437.080 -25570.382, -172473.42..."


0:10:33.561946


In [6]:
start = datetime.datetime.now()

SELECTED_DATE = "2021-10-15"
INCLUDE_ITP = [4, 300, 182]

gdf = shared_utils.geography_utils.make_routes_gdf(SELECTED_DATE, 
                                                   CRS = "EPSG:2229", 
                                                   ITP_ID_LIST = INCLUDE_ITP)
display(gdf.head())

end = datetime.datetime.now()
print(end - start)

,calitp_itp_id,calitp_url_number,shape_id,pt_array,geometry
0,4,0,shp-232-58,"[POINT(-122.023722 37.580733), POINT(-122.0236...","LINESTRING (5394574.734 3149550.553, 5394588.6..."
1,4,0,shp-649-58,"[POINT(-122.125981 37.753012), POINT(-122.1257...","LINESTRING (5367508.189 3213510.856, 5367572.7..."
2,4,0,shp-215-67,"[POINT(-121.97561 37.557912), POINT(-121.97574...","LINESTRING (5408193.474 3140682.377, 5408152.5..."
3,4,0,shp-669-04,"[POINT(-122.340413 37.970909), POINT(-122.3403...","LINESTRING (5308914.097 3295510.119, 5308933.5..."
4,4,0,shp-OX-53,"[POINT(-122.24383 37.762955), POINT(-122.24374...","LINESTRING (5333572.853 3218553.129, 5333599.8..."


0:00:47.867253
